In [1]:
pwd

'/tf/workspace/src'

In [2]:
MGF_FILE = "../datasets/small.mgf"
MZID_FILE = "../datasets/PXD010000/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_msgfplus.mzid"
PARQUET_FILE = "../dumps/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39.parquet"
PADDING_CHARACTER = '_'
PADDING_VALUE = 0

In [3]:
STATS_FILE = '.'.join(PARQUET_FILE.split('.')[:-1]) + '_stats.json'
STATS_FILE

'../dumps/Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun16_Pippin_16-03-39_stats.json'

In [4]:
from pyteomics import parser, mgf, mzid
import pandas as pd
import numpy as np
import json

In [5]:
from utils.formats import read

In [6]:
mgf_id = read(MGF_FILE)
mgf_id

<class 'pyteomics.mgf.IndexedMGF'>
Length: 71
Example:

{'params': {'pepmass': (786.8843383789, None), 'charge': [2], 'mslevel': '2', 'collision_energy': '0.0', 'filename': 'filtered_library_mgf_files/6ba3f3d0fc82436db3e7f59cd561043c.mgf', 'seq': 'AAAGEEETAAAGSPGRK', 'protein': 'sp|Q9NS37|ZHANG_HUMAN', 'scans': '1', 'title': '1', 'score': '15.986790441', 'fdr': '0.0'}, 'm/z array': array([ 115.08673859,  129.06513977,  143.08161926,  169.09707642,
        187.07124329,  200.10276794,  201.08705139,  214.11856079,
        226.11781311,  227.10159302,  228.09733582,  241.0809021 ,
        243.14474487,  258.10855103,  259.09121704,  271.14016724,
        280.12808228,  284.1222229 ,  298.10293579,  298.13931274,
        301.14950562,  311.13476562,  316.11343384,  325.15057373,
        329.14501953,  329.1918335 ,  330.13134766,  342.13040161,
        354.17675781,  355.16122437,  359.15740967,  369.14044189,
        372.18493652,  382.1723938 ,  387.15136719,  388.13961792,
        395.

,pepmass,charge,mslevel,collision_energy,filename,seq,protein,scans,title,score,fdr,m/z array,intensity array,charge array
0,"(786.8843383789, None)",[2+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AAAGEEETAAAGSPGRK,sp|Q9NS37|ZHANG_HUMAN,1,1,15.986790441,0.0,"[115.08673858642578, 129.0651397705078, 143.08...","[93472.9609375, 34148.0, 241092.0625, 68735.73...","[--, --, --, --, --, --, --, --, --, --, --, -..."
1,"(645.8395385742, None)",[2+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AAALASGC+57.021TVEIK,sp|Q8IYS1|P20D2_HUMAN,2,2,11.5246829423,0.0,"[115.08673095703125, 129.10223388671875, 130.0...","[408930.21875, 145201.15625, 196776.15625, 136...","[--, --, --, --, --, --, --, --, --, --, --, -..."
2,"(893.4879760742, None)",[2+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AAAVLRDSTSVPVTAEAK,sp|Q9BWM5|ZN416_HUMAN,3,3,8.363003655,0.0,"[129.10223388671875, 130.08627319335938, 147.1...","[114174.90625, 46108.5625, 134812.828125, 4005...","[--, --, --, --, --, --, --, --, --, --, --, -..."
3,"(641.6273193359, None)",[3+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AADFLFSC+57.021DASHPDTLR,sp|Q8NHR7|TERB2_HUMAN,4,4,14.6525929701,0.0,"[115.08660125732422, 120.08078002929688, 129.1...","[89966.0, 132922.90625, 20467.626953125, 18817...","[--, --, --, --, --, --, --, --, --, --, --, -..."
4,"(752.6887207031, None)",[3+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AADSSAPEDSEKLVGDTVSYSK,sp|Q08357|S20A2_HUMAN,5,5,21.4883967892,0.0,"[115.08673095703125, 129.1023712158203, 130.08...","[411124.71875, 163406.703125, 32186.740234375,...","[--, --, --, --, --, --, --, --, --, --, --, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,"(439.2272949219, None)",[3+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AGNVSSRSGEPVR,sp|Q14549|GBX1_HUMAN,67,67,10.6831862643,0.0,"[101.07078552246094, 110.07115173339844, 115.0...","[61126.3984375, 57858.921875, 22904.6875, 2358...","[--, --, --, --, --, --, --, --, --, --, --, -..."
67,"(635.8969116211, None)",[2+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AGPVLGLVLYLR,,68,68,8.3080313664,0.0,"[120.08051300048828, 169.1332244873047, 175.11...","[16868.681640625, 105248.5859375, 34479.578125...","[--, --, --, --, --, --, --, --, --, --, --, -..."
68,"(509.5815124512, None)",[3+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AGRGPGQNDLAGEER,,69,69,13.1649538986,0.0,"[127.08671569824219, 129.06602478027344, 129.1...","[135893.25, 236149.625, 90846.671875, 155520.4...","[--, --, --, --, --, --, --, --, --, --, --, -..."
69,"(939.4699707031, None)",[3+],2,0.0,filtered_library_mgf_files/6ba3f3d0fc82436db3e...,AGSHISSNRAYDFLYDPLFIVSSEK,sp|Q7Z4T9|MAAT1_HUMAN,70,70,13.3951726171,0.0,"[110.070556640625, 120.08097076416016, 129.101...","[32837.21484375, 53190.16796875, 25042.9042968...","[--, --, --, --, --, --, --, --, --, --, --, -..."


In [7]:
mzid_df = read(MZID_FILE)
mzid_df

<class 'pyteomics.mzid.MzIdentML'>
Length: 29447
Example:

{
    "spectrumID": "controllerType=0 controllerNumber=1 scan=31013",
    "SpectrumIdentificationItem": [
        {
            "passThreshold": true,
            "rank": 1,
            "calculatedMassToCharge": 1110.07861328125,
            "experimentalMassToCharge": 1110.32568359375,
            "chargeState": 4,
            "PeptideEvidenceRef": [
                {
                    "isDecoy": false,
                    "post": "T",
                    "pre": "R",
                    "end": 87,
                    "start": 47,
                    "PeptideSequence": "EASLPSLSEITITKEYDVASGDLLKEALSGNVQEVDIVFTR",
                    "accession": "ref|YP_001236175.1",
                    "length": 161,
                    "protein description": "ref|YP_001236175.1 hypothetical protein Acry_3068 (gi|148262048)",
                    "numDatabaseSequences": 7150,
                    "location": "C:\\DMS_Temp_Org\\ID_005061_A9F312

,spectrumID,scan number(s),scan start time,location,name,FileFormat,SpectrumIDFormat,passThreshold,rank,calculatedMassToCharge,...,start,accession,length,protein description,numDatabaseSequences,ID_005061_A9F31275.fasta,DB composition target+decoy,decoy DB accession regexp,decoy DB type reverse,Modification
0,controllerType=0 controllerNumber=1 scan=31013,31013.0,83.444176,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,1110.078613,...,47,ref|YP_001236175.1,161,ref|YP_001236175.1 hypothetical protein Acry_3...,7150,,,^XXX,,NaN
1,controllerType=0 controllerNumber=1 scan=28498,28498.0,76.036940,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,1049.326050,...,75,ref|YP_001235217.1,123,"ref|YP_001235217.1 cytochrome c, class I (gi|1...",7150,,,^XXX,,"[{'monoisotopicMassDelta': 15.99491463, 'locat..."
2,controllerType=0 controllerNumber=1 scan=29226,29226.0,78.162450,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,982.011353,...,2,ref|YP_001234892.1,149,ref|YP_001234892.1 MucR family transcriptional...,7150,,,^XXX,,NaN
3,controllerType=0 controllerNumber=1 scan=34878,34878.0,95.003060,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,943.780640,...,166,ref|YP_001235065.1,216,ref|YP_001235065.1 50S ribosomal protein L4 (g...,7150,,,^XXX,,NaN
4,controllerType=0 controllerNumber=1 scan=33699,33699.0,91.475280,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,911.251831,...,286,ref|YP_001235613.1,549,ref|YP_001235613.1 chaperonin GroEL (gi|148261...,7150,,,^XXX,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29442,controllerType=0 controllerNumber=1 scan=2115,2115.0,8.095779,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,800.561218,...,258,XXX_ref|YP_001234263.1,504,NaN,7150,,,^XXX,,"[{'monoisotopicMassDelta': 15.99491463, 'locat..."
29443,controllerType=0 controllerNumber=1 scan=34773,34773.0,94.695150,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,903.816223,...,570,XXX_ref|YP_001234065.1,686,NaN,7150,,,^XXX,,"[{'monoisotopicMassDelta': 15.99491463, 'locat..."
29444,controllerType=0 controllerNumber=1 scan=7061,7061.0,20.723070,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,904.671021,...,471,XXX_ref|YP_001233341.1,579,NaN,7150,,,^XXX,,NaN
29445,controllerType=0 controllerNumber=1 scan=1971,1971.0,7.665452,C:\DMS_WorkDir8\Biodiversity_A_cryptum_FeTSB_a...,Biodiversity_A_cryptum_FeTSB_anaerobic_1_01Jun...,mzML file,Thermo nativeID format,True,1,637.858459,...,70,XXX_Contaminant_K1C9_HUMAN,623,NaN,7150,,,^XXX,,NaN


In [8]:
mzid_df.columns

Index(['spectrumID', 'scan number(s)', 'scan start time', 'location', 'name',
       'FileFormat', 'SpectrumIDFormat', 'passThreshold', 'rank',
       'calculatedMassToCharge', 'experimentalMassToCharge', 'chargeState',
       'MS-GF:RawScore', 'MS-GF:DeNovoScore', 'MS-GF:SpecEValue',
       'MS-GF:EValue', 'MS-GF:QValue', 'MS-GF:PepQValue', 'IsotopeError',
       'AssumedDissociationMethod', 'ExplainedIonCurrentRatio',
       'NTermIonCurrentRatio', 'CTermIonCurrentRatio', 'MS2IonCurrent',
       'NumMatchedMainIons', 'MeanErrorAll', 'StdevErrorAll', 'MeanErrorTop7',
       'StdevErrorTop7', 'MeanRelErrorAll', 'StdevRelErrorAll',
       'MeanRelErrorTop7', 'StdevRelErrorTop7', 'PeptideSequence', 'isDecoy',
       'post', 'pre', 'end', 'start', 'accession', 'length',
       'protein description', 'numDatabaseSequences',
       'ID_005061_A9F31275.fasta', 'DB composition target+decoy',
       'decoy DB accession regexp', 'decoy DB type reverse', 'Modification'],
      dtype='object')

In [ ]:
# drop non-AA characters
mgf_df.seq = mgf_df.seq.str.replace(r"[^A-Z]",'')

In [ ]:
stats = dict()

In [ ]:
for col in mgf_df.columns:
    stats[col] = dict()

In [ ]:
for col in mgf_df.columns:
    stats[col]['max_length'] = mgf_df[col].str.len().max()
stats

In [ ]:
for col in ['mz', 'intensities']:
    stats[col]['min_value'] = mgf_df[col].apply(min).min()
    stats[col]['max_value'] = mgf_df[col].apply(max).max()
stats

In [ ]:
# pad sequences with padding char
for col in ['seq']:
    mgf_df[col] = mgf_df[col].str.pad(width=stats[col]['max_length'], side='right', fillchar=PADDING_CHARACTER)
    assert mgf_df[col].str.len().min() == mgf_df[col].str.len().max()

In [ ]:
# pad value columns with zeros
for col in ['mz', 'intensities']:
    mgf_df[col] = pd.DataFrame.from_records(mgf_df[col]).fillna(PADDING_VALUE).values.tolist()
    assert mgf_df[col].str.len().min() == mgf_df[col].str.len().max()

In [ ]:
mgf_df

In [ ]:
alphabet = list(PADDING_CHARACTER)
alphabet += parser.std_amino_acids
stats['alphabet'] = alphabet
stats['padding_character'] = PADDING_CHARACTER
stats

In [ ]:
aa_to_idx = {aa:idx for idx, aa in enumerate(stats['alphabet'])}
idx_to_aa = {idx:aa for aa, idx in aa_to_idx.items()}
stats['aa_to_idx'] = aa_to_idx
stats['idx_to_aa'] = idx_to_aa
stats

In [ ]:
stats

In [ ]:
stats = denumpyfy_dict(stats)
stats

In [ ]:
with open(STATS_FILE, 'w') as file:
    json.dump(stats, file, indent=4)

In [ ]:
mgf_df.to_parquet(PARQUET_FILE, engine='pyarrow')

In [ ]:
mgf_df = pd.read_parquet(PARQUET_FILE)
mgf_df

In [ ]:
with open(STATS_FILE, 'r') as file:
    stats = json.load(file)
stats

In [ ]:
# merge_mz_and_intensities
mgf_df['mz_intensities'] = mgf_df.apply(lambda row: pd.DataFrame(zip(row['mz'], row['intensities'])).values, axis=1)
mgf_df

In [ ]:
mgf_df.mz_intensities.apply(np.shape)

In [ ]:
mgf_df = mgf_df.drop(columns=['mz', 'intensities'])
mgf_df